In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
import keras
import cv2
#from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#import pylab
#plt.rcParams['figure.figsize'] = 10, 10
#%matplotlib inline

train = pd.read_json("kaggle_lceberg_data/train.json")
target_train=train['is_iceberg']
test = pd.read_json("kaggle_lceberg_data/test.json")

target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
#X_band_3=(X_band_1+X_band_2)/2
X_band_3=np.fabs(np.subtract(X_band_1,X_band_2))
X_band_4=np.maximum(X_band_1,X_band_2)
X_band_5=np.minimum(X_band_1,X_band_2)
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])
X_train = np.concatenate([
                          
                          X_band_3[:, :, :, np.newaxis],X_band_4[:, :, :, np.newaxis],X_band_5[:, :, :, np.newaxis]], axis=-1)



X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
#X_band_test_3=(X_band_test_1+X_band_test_2)/2
X_band_test_3=np.fabs(np.subtract(X_band_test_1,X_band_test_2))
X_band_test_4=np.maximum(X_band_test_1,X_band_test_2)
X_band_test_5=np.minimum(X_band_test_1,X_band_test_2)
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([
                          X_band_test_3[:, :, :, np.newaxis], X_band_test_4[:, :, :, np.newaxis],X_band_test_5[:, :, :, np.newaxis]],axis=-1)
RESIZE = None
# #ResNet
# RESIZE = 197
# #InceptionV3
# RESIZE = 139
#Xception
# RESIZE = 71
if RESIZE != None:
    X_train_resize = np.zeros((X_train.shape[0], RESIZE, RESIZE, 3), np.float32)
    X_test_resize = np.zeros((X_test.shape[0], RESIZE, RESIZE, 3), np.float32)
    # X_train_resize[:,...] = cv2.resize(X_train[:,...], (197, 197))
    # print "train data set resize done!"
    # X_test_resize[:,...] = cv2.resize(X_test[:,...], (197, 197))
    # print "test data set resize done!"
    for i in range(X_train.shape[0]):
    #     a = X_train[i]
        X_train_resize[i] = cv2.resize(X_train[i], (RESIZE, RESIZE))
    #     X_train[i] = a
    print "train data set resize done!"
    for i in range(X_test.shape[0]):
    #     a = X_test[i]
        X_test_resize[i] = cv2.resize(X_test[i], (RESIZE, RESIZE))
    #     X_test[i] = a
    print "test data set resize done!"
#Import Keras.
#from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator
batch_size=32
PARA_PATH = './VGG16/'
# Define the image transformations here
# gen = ImageDataGenerator(horizontal_flip = True,
#                          vertical_flip = True,
#                          width_shift_range = 0.,
#                          height_shift_range = 0.,
#                          channel_shift_range=0,
#                          zoom_range = 0.5,
#                          rotation_range = 10)
gen = ImageDataGenerator(width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.2,
                         rotation_range = 20,
                         horizontal_flip = True)
# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=2):
   #es = EarlyStopping('val_loss', patience=10, mode="min")
   es = EarlyStopping('val_loss', patience=20, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]


def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    x = GlobalMaxPooling2D()(x)
#     base_model2 = keras.applications.mobilenet.MobileNet(weights=None, alpha=0.9,input_tensor = base_model.input,include_top=False, input_shape=X_train.shape[1:])

#     x2 = base_model2.output
#     x2 = GlobalAveragePooling2D()(x2)

#     merge_one = concatenate([x, x2, angle_layer])
    merge_one = concatenate([x, angle_layer])
    merge_one = Dropout(0.5)(merge_one)
    
#     dense_layer = Dropout( 0.5 ) ( BatchNormalization(momentum=0.99) (Dense(4096, activation = 'relu') (merge_one)) )
#     merge_one = Dropout( 0.5 ) ( BatchNormalization(momentum=bn_momentum) (Dense(4096, activation = 'relu') (dense_layer)) )
    
    predictions = Dense(1, activation='sigmoid',kernel_initializer='he_normal')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = Adam(lr=1e-4) #SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model

def getResNet50AngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = ResNet50(weights='imagenet', include_top=False, 
                 input_shape=X_train_resize.shape[1:], classes=1)
    x = base_model.get_layer('avg_pool').output
    x = GlobalMaxPooling2D()(x)
#     x = GlobalAveragePooling2D()(x)
#     base_model2 = keras.applications.mobilenet.MobileNet(weights=None, alpha=0.9,input_tensor = base_model.input,include_top=False, input_shape=X_train.shape[1:])

#     x2 = base_model2.output
#     x2 = GlobalAveragePooling2D()(x2)

#     merge_one = concatenate([x, x2, angle_layer])
    merge_one = concatenate([x, angle_layer])
    merge_one = Dropout(0.5)(merge_one)
    
#     dense_layer = Dropout( 0.5 ) ( BatchNormalization(momentum=0.99) (Dense(4096, activation = 'relu') (merge_one)) )
#     merge_one = Dropout( 0.5 ) ( BatchNormalization(momentum=bn_momentum) (Dense(4096, activation = 'relu') (dense_layer)) )
    
    predictions = Dense(1, activation='sigmoid',kernel_initializer='he_normal')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = Adam(lr=1e-4) #SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    model.summary()
    return model

def InceptionV3Model():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = InceptionV3(weights='imagenet', include_top=False, 
                 input_shape=X_train_resize.shape[1:], classes=1)
    x = base_model.get_layer('mixed10').output
    x = GlobalMaxPooling2D()(x)
#     base_model2 = keras.applications.mobilenet.MobileNet(weights=None, alpha=0.9,input_tensor = base_model.input,include_top=False, input_shape=X_train.shape[1:])

#     x2 = base_model2.output
#     x2 = GlobalAveragePooling2D()(x2)

#     merge_one = concatenate([x, x2, angle_layer])
    merge_one = concatenate([x, angle_layer])
    merge_one = Dropout(0.5)(merge_one)
    
#     dense_layer = Dropout( 0.5 ) ( BatchNormalization(momentum=0.99) (Dense(4096, activation = 'relu') (merge_one)) )
#     merge_one = Dropout( 0.5 ) ( BatchNormalization(momentum=bn_momentum) (Dense(4096, activation = 'relu') (dense_layer)) )
    
    predictions = Dense(1, activation='sigmoid',kernel_initializer='he_normal')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = Adam(lr=1e-4) #SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    model.summary()
    return model

def XceptionModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = Xception(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block14_sepconv2_act').output
    x = GlobalMaxPooling2D()(x)
#     base_model2 = keras.applications.mobilenet.MobileNet(weights=None, alpha=0.9,input_tensor = base_model.input,include_top=False, input_shape=X_train.shape[1:])

#     x2 = base_model2.output
#     x2 = GlobalAveragePooling2D()(x2)

#     merge_one = concatenate([x, x2, angle_layer])
    merge_one = concatenate([x, angle_layer])
    merge_one = Dropout(0.5)(merge_one)
    
#     dense_layer = Dropout( 0.5 ) ( BatchNormalization(momentum=0.99) (Dense(4096, activation = 'relu') (merge_one)) )
#     merge_one = Dropout( 0.5 ) ( BatchNormalization(momentum=bn_momentum) (Dense(4096, activation = 'relu') (dense_layer)) )
    
    predictions = Dense(1, activation='sigmoid',kernel_initializer='he_normal')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = Adam(lr=1e-4) #SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    model.summary()
    return model

def MobileNetModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = MobileNet(weights='imagenet', include_top=False, 
                 input_shape=X_train_resize.shape[1:], classes=1)
    x = base_model.get_layer('conv_pw_13_bn').output
    x = GlobalMaxPooling2D()(x)
#     base_model2 = keras.applications.mobilenet.MobileNet(weights=None, alpha=0.9,input_tensor = base_model.input,include_top=False, input_shape=X_train.shape[1:])

#     x2 = base_model2.output
#     x2 = GlobalAveragePooling2D()(x2)

#     merge_one = concatenate([x, x2, angle_layer])
    merge_one = concatenate([x, angle_layer])
    merge_one = Dropout(0.5)(merge_one)
    
#     dense_layer = Dropout( 0.5 ) ( BatchNormalization(momentum=0.99) (Dense(4096, activation = 'relu') (merge_one)) )
#     merge_one = Dropout( 0.5 ) ( BatchNormalization(momentum=bn_momentum) (Dense(4096, activation = 'relu') (dense_layer)) )
    
    predictions = Dense(1, activation='sigmoid',kernel_initializer='he_normal')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = Adam(lr=1e-4) #SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    model.summary()
    return model
#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=5
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log = 0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = PARA_PATH + "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=10)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
#         galaxyModel= getResNet50AngleModel()
#         galaxyModel= InceptionV3Model()
#         galaxyModel= XceptionModel()
#         galaxyModel= MobileNetModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

# preds=myAngleCV(X_train_resize, X_angle, X_test_resize)
preds=myAngleCV(X_train, X_angle, X_test)
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv(PARA_PATH + 'sub201801141429VGG16.csv', index=False)

('\n===================FOLD=', 0)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:160: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=[<tf.Tenso...)`


Epoch 1/100
24/24 [==============================] - 14s 588ms/step - loss: 0.7869 - acc: 0.5638 - val_loss: 0.6134 - val_acc: 0.6335
Epoch 2/100
24/24 [==============================] - 2s 96ms/step - loss: 0.7212 - acc: 0.5443 - val_loss: 0.5694 - val_acc: 0.5311
Epoch 3/100
24/24 [==============================] - 2s 95ms/step - loss: 0.6221 - acc: 0.5729 - val_loss: 0.5108 - val_acc: 0.7112
Epoch 4/100
24/24 [==============================] - 2s 76ms/step - loss: 0.7015 - acc: 0.5483 - val_loss: 0.7120 - val_acc: 0.4689
Epoch 5/100
24/24 [==============================] - 2s 77ms/step - loss: 0.7197 - acc: 0.5052 - val_loss: 0.6949 - val_acc: 0.4814: 0s - loss: 0.7255 -
Epoch 6/100
24/24 [==============================] - 2s 74ms/step - loss: 0.7166 - acc: 0.4987 - val_loss: 0.6919 - val_acc: 0.5155
Epoch 7/100
24/24 [==============================] - 2s 74ms/step - loss: 0.7151 - acc: 0.4974 - val_loss: 0.6903 - val_acc: 0.5466
Epoch 8/100
24/24 [==============================] - 

24/24 [==============================] - 2s 77ms/step - loss: 0.1993 - acc: 0.9165 - val_loss: 0.2078 - val_acc: 0.9128
Epoch 39/100
24/24 [==============================] - 2s 78ms/step - loss: 0.2469 - acc: 0.9023 - val_loss: 0.1636 - val_acc: 0.9346
Epoch 40/100
24/24 [==============================] - 2s 75ms/step - loss: 0.2006 - acc: 0.9139 - val_loss: 0.1653 - val_acc: 0.9377
Epoch 41/100
24/24 [==============================] - 2s 96ms/step - loss: 0.3075 - acc: 0.8927 - val_loss: 0.2042 - val_acc: 0.9283
Epoch 42/100
24/24 [==============================] - 2s 77ms/step - loss: 0.3851 - acc: 0.8138 - val_loss: 0.2202 - val_acc: 0.9159
Epoch 43/100
24/24 [==============================] - 2s 77ms/step - loss: 0.2726 - acc: 0.8823 - val_loss: 0.2500 - val_acc: 0.9159
Epoch 44/100
24/24 [==============================] - 2s 77ms/step - loss: 0.2384 - acc: 0.9010 - val_loss: 0.1834 - val_acc: 0.9252
Epoch 45/100
24/24 [==============================] - 2s 75ms/step - loss: 0.3215 

24/24 [==============================] - 2s 94ms/step - loss: 1.2231 - acc: 0.6667 - val_loss: 0.5302 - val_acc: 0.7321
Epoch 4/100
24/24 [==============================] - 2s 92ms/step - loss: 1.0977 - acc: 0.7014 - val_loss: 0.3592 - val_acc: 0.8629
Epoch 5/100
24/24 [==============================] - 2s 103ms/step - loss: 1.0449 - acc: 0.7331 - val_loss: 0.3525 - val_acc: 0.8255
Epoch 6/100
24/24 [==============================] - 2s 80ms/step - loss: 0.8313 - acc: 0.7753 - val_loss: 0.3939 - val_acc: 0.8567
Epoch 7/100
24/24 [==============================] - 2s 80ms/step - loss: 0.8274 - acc: 0.7541 - val_loss: 0.8278 - val_acc: 0.6511
Epoch 8/100
24/24 [==============================] - 2s 96ms/step - loss: 0.7730 - acc: 0.7565 - val_loss: 0.2459 - val_acc: 0.9034
Epoch 9/100
24/24 [==============================] - 2s 80ms/step - loss: 0.5648 - acc: 0.8252 - val_loss: 0.2979 - val_acc: 0.8910
Epoch 10/100
24/24 [==============================] - 2s 78ms/step - loss: 0.6879 - acc

Epoch 65/100
24/24 [==============================] - 2s 75ms/step - loss: 0.1819 - acc: 0.9257 - val_loss: 0.1874 - val_acc: 0.9377
Epoch 66/100
24/24 [==============================] - 2s 77ms/step - loss: 0.1695 - acc: 0.9258 - val_loss: 0.2078 - val_acc: 0.9315
('Train loss:', 0.14219989391248483)
('Train accuracy:', 0.93764614185502726)
('Test loss:', 0.15553328580574083)
('Test accuracy:', 0.93146417445482865)
('\n===================FOLD=', 3)
Epoch 1/100
24/24 [==============================] - 14s 588ms/step - loss: 1.4362 - acc: 0.5221 - val_loss: 1.4434 - val_acc: 0.5312
Epoch 2/100
24/24 [==============================] - 2s 98ms/step - loss: 1.0915 - acc: 0.5725 - val_loss: 0.7460 - val_acc: 0.6312
Epoch 3/100
24/24 [==============================] - 2s 96ms/step - loss: 0.8377 - acc: 0.6341 - val_loss: 0.5226 - val_acc: 0.7531
Epoch 4/100
24/24 [==============================] - 2s 76ms/step - loss: 0.9002 - acc: 0.6461 - val_loss: 0.6926 - val_acc: 0.7031
Epoch 5/100
24/2

Epoch 3/100
24/24 [==============================] - 2s 78ms/step - loss: 1.0519 - acc: 0.6797 - val_loss: 0.5471 - val_acc: 0.7906
Epoch 4/100
24/24 [==============================] - 2s 90ms/step - loss: 1.0610 - acc: 0.7165 - val_loss: 0.3956 - val_acc: 0.8000
Epoch 5/100
24/24 [==============================] - 2s 93ms/step - loss: 0.7756 - acc: 0.7617 - val_loss: 0.3882 - val_acc: 0.8531
Epoch 6/100
24/24 [==============================] - 2s 92ms/step - loss: 0.8576 - acc: 0.7578 - val_loss: 0.3715 - val_acc: 0.8406
Epoch 7/100
24/24 [==============================] - 2s 91ms/step - loss: 0.6852 - acc: 0.8161 - val_loss: 0.3518 - val_acc: 0.8438
Epoch 8/100
24/24 [==============================] - 2s 93ms/step - loss: 0.6053 - acc: 0.8008 - val_loss: 0.2945 - val_acc: 0.8625
Epoch 9/100
24/24 [==============================] - 2s 75ms/step - loss: 0.5407 - acc: 0.8292 - val_loss: 0.4127 - val_acc: 0.8562
Epoch 10/100
24/24 [==============================] - 2s 78ms/step - loss: 0